In [1]:
import time

import numpy as np
import pandas as pd
import torch
from torch import nn
from tqdm import tqdm

# random seed
np.random.seed(42)
torch.manual_seed(42)
torch.cuda.manual_seed(42)

In [2]:
# turn off warning
import logging
logging.getLogger('pyomo.core').setLevel(logging.ERROR)

## Problem Setting

In [3]:
# init
steepness = 50    # steepness factor
num_blocks = 1    # number of expression blocks
num_data = 5000   # number of data
test_size = 1000  # number of test size
val_size = 1000   # number of validation size
train_size = num_data - test_size - val_size

In [4]:
# parameters as input data
p_low, p_high = 1.0, 8.0
a_low, a_high = 0.5, 4.5
p_train = np.random.uniform(p_low, p_high, (train_size, 1)).astype(np.float32)
p_test  = np.random.uniform(p_low, p_high, (test_size, 1)).astype(np.float32)
p_dev   = np.random.uniform(p_low, p_high, (val_size, 1)).astype(np.float32)
a_train = np.random.uniform(a_low, a_high, (train_size, num_blocks)).astype(np.float32)
a_test  = np.random.uniform(a_low, a_high, (test_size, num_blocks)).astype(np.float32)
a_dev   = np.random.uniform(a_low, a_high, (val_size, num_blocks)).astype(np.float32)

In [5]:
# nm datasets
from neuromancer.dataset import DictDataset
data_train = DictDataset({"p":p_train, "a":a_train}, name="train")
data_test = DictDataset({"p":p_test, "a":a_test}, name="test")
data_dev = DictDataset({"p":p_dev, "a":a_dev}, name="dev")
# torch dataloaders
from torch.utils.data import DataLoader
loader_train = DataLoader(data_train, batch_size=32, num_workers=0, collate_fn=data_train.collate_fn, shuffle=True)
loader_test = DataLoader(data_test, batch_size=32, num_workers=0, collate_fn=data_test.collate_fn, shuffle=False)
loader_dev = DataLoader(data_dev, batch_size=32, num_workers=0, collate_fn=data_dev.collate_fn, shuffle=True)

## Exact Solver

In [6]:
from src.problem import msRosenbrock
model = msRosenbrock(steepness, num_blocks, timelimit=60)

In [7]:
params, sols, objvals, conviols, elapseds = [], [], [], [], []
for p, a in tqdm(list(zip(p_test, a_test))):
    # set params
    model.set_param_val({"p":p, "a":a})
    # solve
    tick = time.time()
    xval, objval = model.solve("scip")
    tock = time.time()
    # eval
    params.append(list(p)+list(a))
    sols.append(list(list(xval.values())[0].values()))
    objvals.append(objval)
    conviols.append(sum(model.cal_violation()))
    elapseds.append(tock - tick)
df = pd.DataFrame({"Param":params, "Sol":sols, "Obj Val": objvals, "Constraints Viol": conviols, "Elapsed Time": elapseds})
df.to_csv("result/rb_exact_50-1.csv")
df = df.iloc[:100]
time.sleep(1)
print(df.describe())
print("Number of infeasible solution: {}".format(np.sum(df["Constraints Viol"] > 0)))

100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [11:49<00:00,  1.41it/s]


          Obj Val  Constraints Viol  Elapsed Time
count  100.000000             100.0    100.000000
mean     1.527909               0.0      0.662830
std      1.889268               0.0      0.253617
min      0.000028               0.0      0.125504
25%      0.166743               0.0      0.402626
50%      0.775518               0.0      0.738829
75%      2.170996               0.0      0.900489
max      9.005967               0.0      1.125116
Number of infeasible solution: 0


## Heuristic - Round

In [8]:
from src.heuristic import naive_round

In [9]:
params, sols, objvals, conviols, elapseds = [], [], [], [], []
for p, a in tqdm(list(zip(p_test, a_test))):
    # set params
    model.set_param_val({"p":p, "a":a})
    # relax
    model_rel = model.relax()
    # solve
    tick = time.time()
    xval_rel, _ = model_rel.solve("scip")
    xval, objval = naive_round(xval_rel, model)
    tock = time.time()
    # eval
    params.append(list(p)+list(a))
    sols.append(list(list(xval.values())[0].values()))
    objvals.append(objval)
    conviols.append(sum(model.cal_violation()))
    elapseds.append(tock - tick)
df = pd.DataFrame({"Param":params, "Sol":sols, "Obj Val": objvals, "Constraints Viol": conviols, "Elapsed Time": elapseds})
time.sleep(1)
df.to_csv("result/rb_heur_rnd_50-1.csv")
df = df.iloc[:100]
time.sleep(1)
print(df.describe())
print("Number of infeasible solution: {}".format(np.sum(df["Constraints Viol"] > 0)))

100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [14:03<00:00,  1.19it/s]


          Obj Val  Constraints Viol  Elapsed Time
count  100.000000        100.000000    100.000000
mean     5.318809          0.023201      0.845810
std      3.815887          0.088962      0.125804
min      0.033126          0.000000      0.415789
25%      1.929421          0.000000      0.788843
50%      4.749954          0.000000      0.800843
75%      7.944151          0.000000      0.906330
max     14.637508          0.467117      1.131778
Number of infeasible solution: 8


## Heuristic - N1

In [10]:
model_heur = model.first_solution_heuristic(nodes_limit=1)

In [11]:
params, sols, objvals, conviols, elapseds = [], [], [], [], []
for p, a in tqdm(list(zip(p_test, a_test))):
    # set params
    model_heur.set_param_val({"p":p, "a":a})
    # solve
    tick = time.time()
    xval, objval = model_heur.solve("scip")
    tock = time.time()
    # eval
    params.append(list(p)+list(a))
    sols.append(list(list(xval.values())[0].values()))
    objvals.append(objval)
    conviols.append(sum(model_heur.cal_violation()))
    elapseds.append(tock - tick)
df = pd.DataFrame({"Param":params, "Sol":sols, "Obj Val": objvals, "Constraints Viol": conviols, "Elapsed Time": elapseds})
time.sleep(1)
df.to_csv("result/rb_heur_n1_50-1.csv")
df = df.iloc[:100]
time.sleep(1)
print(df.describe())
print("Number of infeasible solution: {}".format(np.sum(df["Constraints Viol"] > 0)))

100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [01:12<00:00, 13.76it/s]


          Obj Val  Constraints Viol  Elapsed Time
count  100.000000             100.0    100.000000
mean    99.761901               0.0      0.067367
std     68.446574               0.0      0.022057
min      9.367027               0.0      0.058092
25%     45.777581               0.0      0.061003
50%     87.586337               0.0      0.061796
75%    130.537029               0.0      0.062925
max    305.764538               0.0      0.173536
Number of infeasible solution: 0


## Learnable Rounding

In [12]:
# random seed
np.random.seed(42)
torch.manual_seed(42)
torch.cuda.manual_seed(42)

In [13]:
# hyperparameters
penalty_weight = 80   # weight of constraint violation penealty
hlayers_sol = 4       # number of hidden layers for solution mapping
hlayers_rnd = 4       # number of hidden layers for solution mapping
hsize = 32            # width of hidden layers for solution mapping
lr = 1e-3             # learning rate
batch_size = 64       # batch size

In [14]:
# set problem
import neuromancer as nm
from src.problem import nmRosenbrock
from src.func.layer import netFC
from src.func import roundGumbelModel
# build neural architecture for the solution map
func = nm.modules.blocks.MLP(insize=num_blocks+1, outsize=2*num_blocks, bias=True,
                             linear_map=nm.slim.maps["linear"],
                             nonlin=nn.ReLU, hsizes=[hsize]*hlayers_sol)
smap = nm.system.Node(func, ["p", "a"], ["x"], name="smap")
# define rounding model
layers_rnd = netFC(input_dim=3*num_blocks+1, hidden_dims=[hsize]*hlayers_rnd, output_dim=2*num_blocks)
rnd = roundGumbelModel(layers=layers_rnd, param_keys=["p", "a"], var_keys=["x"],  output_keys=["x_rnd"], 
                       int_ind=model.int_ind, continuous_update=True, name="round")
# build neuromancer problem for rounding
components = nn.ModuleList([smap, rnd]).to("cuda")
loss_fn = nmRosenbrock(["p", "a", "x_rnd"], steepness, num_blocks, penalty_weight)

In [15]:
from src.problem.neuromancer.trainer import trainer
# training
epochs = 200                    # number of training epochs
warmup = 20                     # number of epochs to wait before enacting early stopping policy
patience = 20                   # number of epochs with no improvement in eval metric to allow before early stopping
optimizer = torch.optim.AdamW(components.parameters(), lr=lr)
# create a trainer for the problem
my_trainer = trainer(components, loss_fn, optimizer, epochs, patience, warmup, device="cuda")
# training for the rounding problem
my_trainer.train(loader_train, loader_dev)

Epoch 0, Validation Loss: 320.04
Epoch 1, Validation Loss: 52.33
Epoch 2, Validation Loss: 36.36
Epoch 3, Validation Loss: 34.51
Epoch 4, Validation Loss: 27.76
Epoch 5, Validation Loss: 29.97
Epoch 6, Validation Loss: 28.45
Epoch 7, Validation Loss: 28.80
Epoch 8, Validation Loss: 28.80
Epoch 9, Validation Loss: 30.70
Epoch 10, Validation Loss: 29.41
Epoch 11, Validation Loss: 28.43
Epoch 12, Validation Loss: 27.18
Epoch 13, Validation Loss: 28.00
Epoch 14, Validation Loss: 28.56
Epoch 15, Validation Loss: 27.88
Epoch 16, Validation Loss: 27.50
Epoch 17, Validation Loss: 24.84
Epoch 18, Validation Loss: 25.88
Epoch 19, Validation Loss: 26.70
Epoch 20, Validation Loss: 23.93
Epoch 21, Validation Loss: 23.91
Epoch 22, Validation Loss: 23.60
Epoch 23, Validation Loss: 23.51
Epoch 24, Validation Loss: 23.89
Epoch 25, Validation Loss: 22.84
Epoch 26, Validation Loss: 22.90
Epoch 27, Validation Loss: 23.22
Epoch 28, Validation Loss: 22.68
Epoch 29, Validation Loss: 21.50
Epoch 30, Validatio

In [16]:
params, sols, objvals, conviols, elapseds = [], [], [], [], []
for p, a in tqdm(list(zip(p_test, a_test))):
    # data point as tensor
    datapoints = {"p": torch.tensor(np.array([p]), dtype=torch.float32).to("cuda"), 
                  "a": torch.tensor(np.array([a]), dtype=torch.float32).to("cuda"),
                  "name": "test"}
    # infer
    components.eval()
    tick = time.time()
    with torch.no_grad():
        for comp in components:
            datapoints.update(comp(datapoints))
    tock = time.time()
    # assign params
    model.set_param_val({"p":p, "a":a})
    # assign vars
    x = datapoints["x_rnd"]
    for i in range(2*num_blocks):
        model.vars["x"][i].value = x[0,i].item()
    # get solutions
    xval, objval = model.get_val()    
    params.append(list(p)+list(a))
    sols.append(list(list(xval.values())[0].values()))
    objvals.append(objval)
    conviols.append(sum(model.cal_violation()))
    elapseds.append(tock - tick)
df = pd.DataFrame({"Param":params, "Sol":sols, "Obj Val": objvals, "Constraints Viol": conviols, "Elapsed Time": elapseds})
time.sleep(1)
df.to_csv("result/rb_lr_50-1.csv")
df = df.iloc[:100]
time.sleep(1)
print(df.describe())
print("Number of infeasible solution: {}".format(np.sum(df["Constraints Viol"] > 0)))

100%|█████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:02<00:00, 334.02it/s]


          Obj Val  Constraints Viol  Elapsed Time
count  100.000000        100.000000    100.000000
mean    22.119247          0.002249      0.002634
std     11.048114          0.012923      0.000987
min      2.790326          0.000000      0.001000
25%     12.229221          0.000000      0.002000
50%     21.444718          0.000000      0.002510
75%     30.391718          0.000000      0.003000
max     46.643310          0.082597      0.010011
Number of infeasible solution: 3


## Learnable Threshold

In [17]:
# random seed
np.random.seed(42)
torch.manual_seed(42)
torch.cuda.manual_seed(42)

In [18]:
# hyperparameters
penalty_weight = 80   # weight of constraint violation penealty
hlayers_sol = 4       # number of hidden layers for solution mapping
hlayers_rnd = 4       # number of hidden layers for solution mapping
hsize = 32            # width of hidden layers for solution mapping
lr = 1e-3             # learning rate
batch_size = 64       # batch size

In [19]:
# set problem
import neuromancer as nm
from src.problem import nmRosenbrock
from src.func.layer import netFC
from src.func import roundThresholdModel
# build neural architecture for the solution map
func = nm.modules.blocks.MLP(insize=num_blocks+1, outsize=2*num_blocks, bias=True,
                             linear_map=nm.slim.maps["linear"],
                             nonlin=nn.ReLU, hsizes=[hsize]*hlayers_sol)
smap = nm.system.Node(func, ["p", "a"], ["x"], name="smap")
# define rounding model
layers_rnd = netFC(input_dim=3*num_blocks+1, hidden_dims=[hsize]*hlayers_rnd, output_dim=2*num_blocks)
rnd = roundThresholdModel(layers=layers_rnd, param_keys=["p", "a"], var_keys=["x"],  output_keys=["x_rnd"], 
                          int_ind=model.int_ind, continuous_update=True, name="round")
# build neuromancer problem for rounding
components = nn.ModuleList([smap, rnd]).to("cuda")
loss_fn = nmRosenbrock(["p", "a", "x_rnd"], steepness, num_blocks, penalty_weight)

In [20]:
from src.problem.neuromancer.trainer import trainer
# training
epochs = 200                    # number of training epochs
warmup = 20                     # number of epochs to wait before enacting early stopping policy
patience = 20                   # number of epochs with no improvement in eval metric to allow before early stopping
optimizer = torch.optim.AdamW(components.parameters(), lr=lr)
# create a trainer for the problem
my_trainer = trainer(components, loss_fn, optimizer, epochs, patience, warmup, device="cuda")
# training for the rounding problem
my_trainer.train(loader_train, loader_dev)

Epoch 0, Validation Loss: 187.43
Epoch 1, Validation Loss: 30.85
Epoch 2, Validation Loss: 25.13
Epoch 3, Validation Loss: 23.88
Epoch 4, Validation Loss: 24.17
Epoch 5, Validation Loss: 23.73
Epoch 6, Validation Loss: 23.26
Epoch 7, Validation Loss: 22.98
Epoch 8, Validation Loss: 22.39
Epoch 9, Validation Loss: 22.09
Epoch 10, Validation Loss: 22.19
Epoch 11, Validation Loss: 22.04
Epoch 12, Validation Loss: 22.53
Epoch 13, Validation Loss: 21.87
Epoch 14, Validation Loss: 21.77
Epoch 15, Validation Loss: 22.12
Epoch 16, Validation Loss: 21.72
Epoch 17, Validation Loss: 22.96
Epoch 18, Validation Loss: 21.60
Epoch 19, Validation Loss: 22.78
Epoch 20, Validation Loss: 21.38
Epoch 21, Validation Loss: 23.08
Epoch 22, Validation Loss: 21.62
Epoch 23, Validation Loss: 21.39
Epoch 24, Validation Loss: 21.33
Epoch 25, Validation Loss: 21.38
Epoch 26, Validation Loss: 21.92
Epoch 27, Validation Loss: 21.23
Epoch 28, Validation Loss: 21.53
Epoch 29, Validation Loss: 20.89
Epoch 30, Validatio

In [21]:
params, sols, objvals, conviols, elapseds = [], [], [], [], []
for p, a in tqdm(list(zip(p_test, a_test))):
    # data point as tensor
    datapoints = {"p": torch.tensor(np.array([p]), dtype=torch.float32).to("cuda"), 
                  "a": torch.tensor(np.array([a]), dtype=torch.float32).to("cuda"),
                  "name": "test"}
    # infer
    components.eval()
    tick = time.time()
    with torch.no_grad():
        for comp in components:
            datapoints.update(comp(datapoints))
    tock = time.time()
    # assign params
    model.set_param_val({"p":p, "a":a})
    # assign vars
    x = datapoints["x_rnd"]
    for i in range(2*num_blocks):
        model.vars["x"][i].value = x[0,i].item()
    # get solutions
    xval, objval = model.get_val()    
    params.append(list(p)+list(a))
    sols.append(list(list(xval.values())[0].values()))
    objvals.append(objval)
    conviols.append(sum(model.cal_violation()))
    elapseds.append(tock - tick)
df = pd.DataFrame({"Param":params, "Sol":sols, "Obj Val": objvals, "Constraints Viol": conviols, "Elapsed Time": elapseds})
time.sleep(1)
df.to_csv("result/rb_lr_50-1.csv")
df = df.iloc[:100]
time.sleep(1)
print(df.describe())
print("Number of infeasible solution: {}".format(np.sum(df["Constraints Viol"] > 0)))

100%|█████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:03<00:00, 256.32it/s]


          Obj Val  Constraints Viol  Elapsed Time
count  100.000000        100.000000    100.000000
mean    19.438295          0.001846      0.003141
std     10.391273          0.010171      0.000840
min      2.838517          0.000000      0.001506
25%     11.042235          0.000000      0.002510
50%     19.526488          0.000000      0.003009
75%     27.423908          0.000000      0.003993
max     41.876709          0.075890      0.005085
Number of infeasible solution: 5


## Parametric Learning Then Rounding

In [22]:
# random seed
np.random.seed(42)
torch.manual_seed(42)
torch.cuda.manual_seed(42)

In [23]:
# hyperparameters
penalty_weight = 80   # weight of constraint violation penealty
hlayers_sol = 4       # number of hidden layers for solution mapping
hsize = 32            # width of hidden layers for solution mapping
lr = 1e-3             # learning rate
batch_size = 64       # batch size

In [28]:
# set problem
import neuromancer as nm
from src.problem import nmQuadratic
from src.func.layer import netFC
# build neural architecture for the solution map
func = nm.modules.blocks.MLP(insize=num_blocks+1, outsize=2*num_blocks, bias=True,
                             linear_map=nm.slim.maps["linear"],
                             nonlin=nn.ReLU, hsizes=[hsize]*hlayers_sol)
smap = nm.system.Node(func, ["p", "a"], ["x"], name="smap")
# build neuromancer problem for rounding
components = nn.ModuleList([smap]).to("cuda")
loss_fn = nmRosenbrock(["p", "a", "x"], steepness, num_blocks, penalty_weight)

In [29]:
from src.problem.neuromancer.trainer import trainer
# training
epochs = 200                    # number of training epochs
warmup = 20                     # number of epochs to wait before enacting early stopping policy
patience = 20                   # number of epochs with no improvement in eval metric to allow before early stopping
optimizer = torch.optim.AdamW(components.parameters(), lr=lr)
# create a trainer for the problem
my_trainer = trainer(components, loss_fn, optimizer, epochs, patience, warmup, device="cuda")
# training for the rounding problem
my_trainer.train(loader_train, loader_dev)

Epoch 0, Validation Loss: 185.27
Epoch 1, Validation Loss: 38.03
Epoch 2, Validation Loss: 19.96
Epoch 3, Validation Loss: 17.08
Epoch 4, Validation Loss: 17.08
Epoch 5, Validation Loss: 17.14
Epoch 6, Validation Loss: 16.97
Epoch 7, Validation Loss: 16.95
Epoch 8, Validation Loss: 17.06
Epoch 9, Validation Loss: 16.86
Epoch 10, Validation Loss: 17.02
Epoch 11, Validation Loss: 17.06
Epoch 12, Validation Loss: 17.19
Epoch 13, Validation Loss: 17.01
Epoch 14, Validation Loss: 16.91
Epoch 15, Validation Loss: 17.14
Epoch 16, Validation Loss: 17.02
Epoch 17, Validation Loss: 17.02
Epoch 18, Validation Loss: 16.91
Epoch 19, Validation Loss: 17.24
Epoch 20, Validation Loss: 16.97
Epoch 21, Validation Loss: 16.88
Epoch 22, Validation Loss: 16.99
Epoch 23, Validation Loss: 17.02
Epoch 24, Validation Loss: 16.98
Epoch 25, Validation Loss: 17.14
Epoch 26, Validation Loss: 17.04
Epoch 27, Validation Loss: 17.16
Epoch 28, Validation Loss: 16.85
Epoch 29, Validation Loss: 16.99
Epoch 30, Validatio

In [34]:
from src.heuristic import naive_round
params, sols, objvals, conviols, elapseds = [], [], [], [], []
for p, a in tqdm(list(zip(p_test, a_test))):
    # data point as tensor
    datapoints = {"p": torch.tensor(np.array([p]), dtype=torch.float32).to("cuda"), 
                  "a": torch.tensor(np.array([a]), dtype=torch.float32).to("cuda"),
                  "name": "test"}
    # infer
    components.eval()
    tick = time.time()
    with torch.no_grad():
        for comp in components:
            datapoints.update(comp(datapoints))
    tock = time.time()
    # assign params
    model.set_param_val({"p":p, "a":a})
    # assign vars
    x = datapoints["x"]
    for i in range(num_blocks*2):
        model.vars["x"][i].value = x[0,i].item()
    # get solutions
    xval_rel, _ = model.get_val()
    xval, objval = naive_round(xval_rel, model)
    params.append(list(p)+list(a))
    sols.append(list(list(xval.values())[0].values()))
    objvals.append(objval)
    conviols.append(sum(model.cal_violation()))
    elapseds.append(tock - tick)
df = pd.DataFrame({"Param":params, "Sol":sols, "Obj Val": objvals, "Constraints Viol": conviols, "Elapsed Time": elapseds})
time.sleep(1)
df.to_csv("result/rb_pr_50-1.csv")
df = df.iloc[:100]
print(df.describe())
print("Number of infeasible solution: {}".format(np.sum(df["Constraints Viol"] > 0)))

100%|█████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:01<00:00, 825.19it/s]


          Obj Val  Constraints Viol  Elapsed Time
count  100.000000        100.000000    100.000000
mean    21.674119          0.070975      0.000634
std     10.356389          0.140274      0.000544
min      2.965154          0.000000      0.000000
25%     13.824205          0.000000      0.000000
50%     20.230306          0.000000      0.000998
75%     29.364229          0.047537      0.001001
max     47.744203          0.467117      0.002043
Number of infeasible solution: 29
